In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su única responsabilidad es crear la tabla de la Capa de Plata (Silver).
# Importamos las funciones y tipos necesarios de PySpark
from pyspark.sql.functions import col, when, lit, ceil, concat_ws, expr, upper, regexp_extract
from pyspark.sql.types import IntegerType, DoubleType, StringType
import re
from functools import reduce as py_reduce
import operator

# 1. --- Configuración de Tablas ---
source_table = "workspace.tecnomundo_data_raw.reporte_de_ventas_por_articulos_2_raw"
output_silver_table = "workspace.tecnomundo_data_processed.reporte_de_ventas_por_articulos_2_cleaned"
quality_issues_table = "workspace.tecnomundo_data_reporting.reporte_de_ventas_por_articulos_2_quality_issues"

print(f"Tabla de Origen (Bronce): {source_table}")
print(f"Tabla de Destino (Plata): {output_silver_table}")
print(f"Tabla de Problemas de Calidad: {quality_issues_table}")


# 2. --- Lectura de la Tabla Cruda ---
print(f"\nLeyendo datos desde '{source_table}'...")
df = spark.table(source_table)


# 3. --- Saneamiento de Nombres y Creación de Vista Temporal ---
print("\nSaneando nombres de columnas y creando una vista temporal...")
rename_expressions = []
for column in df.columns:
    new_column_name = column.replace('Nº', 'num').replace('º', '').replace('ñ', 'n').replace('Ñ', 'N')
    new_column_name = re.sub(r'[\s\.\-]+', '_', new_column_name)
    new_column_name = re.sub(r'[^a-zA-Z0-9_]', '', new_column_name).lower()
    rename_expressions.append(f"`{column}` as `{new_column_name}`")

df_sanitized = df.selectExpr(*rename_expressions)
df_sanitized.createOrReplaceTempView("ventas_sanitized_temp_view")
print("Vista temporal 'ventas_sanitized_temp_view' creada con nombres de columna limpios.")


# 4. --- Lógica de Limpieza y Validación de Tipos ---
print("\nIniciando el proceso de limpieza y validación de tipos...")
df_to_clean = spark.table("ventas_sanitized_temp_view")

# a) Limpieza de la columna 'fecha'
date_formats = ["dd-MM-yyyy", "M/d/yyyy", "yyyy-MM-dd", "d-M-yyyy"]
coalesce_expr_str = "coalesce(" + ", ".join([f"try_to_timestamp(fecha, '{f}')" for f in date_formats]) + ")"
df_cleaned = df_to_clean.withColumn("fecha_temp", expr(coalesce_expr_str))
df_cleaned = df_cleaned.withColumn("fecha_is_valid", col("fecha_temp").isNotNull())
df_cleaned = df_cleaned.withColumn("fecha", 
    when(col("fecha_is_valid"), col("fecha_temp"))
    .otherwise(lit("1900-01-01").cast("timestamp"))
).drop("fecha_temp")

# b) Limpieza de columnas numéricas
numeric_cols = ["cantidad", "precio_un_", "ganancia", "subtotal"]
for c in numeric_cols:
    df_cleaned = df_cleaned.withColumn(f"{c}_is_valid", col(c).cast(DoubleType()).isNotNull())
    df_cleaned = df_cleaned.withColumn(c, 
        when(col(f"{c}_is_valid"), ceil(col(c).cast(DoubleType())).cast(IntegerType()))
        .otherwise(lit(0))
    )

# c) Limpieza de columnas de texto
text_cols = [c for c in df_to_clean.columns if c not in numeric_cols and c != "fecha"]
for c in text_cols:
    df_cleaned = df_cleaned.withColumn(c, 
        when(col(c).isNull() | (col(c) == ""), lit("Sin registro"))
        .otherwise(col(c))
    )

print("Proceso de limpieza completado.")


# 5. --- Separación de Filas con Problemas de Calidad ---
print("\nIdentificando y separando filas con problemas de calidad...")
conditions = [col(f"{c}_is_valid") == False for c in ["fecha"] + numeric_cols]
final_condition = py_reduce(operator.or_, conditions)
df_quality_issues = df_cleaned.filter(final_condition)
df_good_data = df_cleaned.filter(~final_condition)

# (El resto de la lógica de separación de errores sigue igual...)
issue_description_cols = [when(col(f"{c}_is_valid") == False, lit(f"Problema de tipo en '{c}'")) for c in ["fecha"] + numeric_cols]
df_quality_issues = df_quality_issues.withColumn("quality_issue_description", concat_ws(", ", *issue_description_cols))
df_quality_issues_final = df_quality_issues.select(*df_to_clean.columns, "quality_issue_description")
print(f"Se encontraron {df_quality_issues_final.count()} filas con problemas de calidad.")
print(f"Se encontraron {df_good_data.count()} filas limpias.")


# 6. --- Preparación Final de la Capa de Plata (Modelado y Estandarización de Claves) ---
print("\nPreparando DataFrame final para la Capa de Plata...")

# Paso 6a: Estandarizar la clave del producto
# Aplicamos las reglas de negocio para obtener la clave conformada.
df_conformed = df_good_data.withColumn("codigo_producto_estandarizado",
    # Primero, convertimos todo a mayúsculas para solucionar la inconsistencia de case.
    upper(
        # Luego, aplicamos la lógica condicional para eliminar prefijos de caja.
        # La expresión regular '^[A-Z]\\d{2}-' busca un patrón como 'A04-'.
        when(col("codigo").rlike("^[A-Z]\\d{2}-"), regexp_extract(col("codigo"), r'-([^-]*)$', 1))
        .otherwise(col("codigo"))
    )
)
print("  - Clave de producto estandarizada (mayúsculas y sin prefijos).")


# Paso 6b: Definir explícitamente las columnas finales
# Usamos la nueva clave estandarizada y eliminamos la original.
final_silver_columns = [
    "fecha",
    "comprobante_num",
    col("codigo_producto_estandarizado").alias("codigo_producto"), # Usamos la nueva clave estandarizada
    "cantidad",
    "precio_un_",
    "ganancia",
    "subtotal"
]

df_silver_final = df_conformed.select(final_silver_columns)
print("  - Esquema final de la Capa de Plata definido y aplicado.")


# 7. --- Guardado de las Tablas Finales ---
print(f"\nGuardando datos limpios en la tabla de la Capa de Plata: '{output_silver_table}'...")
(df_silver_final.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(output_silver_table))
print("¡Tabla de la Capa de Plata guardada exitosamente!")

if df_quality_issues_final.count() > 0:
    print(f"Guardando filas con problemas en la tabla '{quality_issues_table}'...")
    (df_quality_issues_final.write
     .mode("overwrite")
     .option("overwriteSchema", "true")
     .saveAsTable(quality_issues_table))
    print("¡Tabla de problemas de calidad guardada exitosamente!")
else:
    print("No se encontraron filas con problemas de calidad para guardar.")

# 8. --- Verificación Final ---
print("\nMostrando una muestra de los datos limpios (Capa de Plata):")
display(spark.table(output_silver_table))
